In [ ]:
## IEX EDA starts here

In [1]:
import os
import requests
import pandas as pd

In [2]:
os.environ.get('IEX_SANDBOX_TOKEN')

'Tsk_2f36139a7da4433bbc4882148386ff32'

In [3]:
base_url = 'https://cloud.iexapis.com/v1'
sandbox_url = 'https://sandbox.iexapis.com/stable'

token = os.environ.get('IEX_TOKEN')
sandbox_token = os.environ.get('IEX_SANDBOX_TOKEN')
params = {'token': token}
sandbox_params = {'token': sandbox_token}


In [130]:
# endpoint = '/stock/aapl/chart'
# endpoint = '/stock/aapl/balance-sheet'
# endpoint = '/stock/aapl/financials'
# endpoint = '/time-series/reported_financials/AAPL/10-Q?last=2'
endpoint = '/stock/aapl/income/10-Q?last=12'
# endpoint = '/time-series'
# endpoint = '/stock/aapl/financials?period=quarter'
resp = requests.get(sandbox_url+endpoint, params = sandbox_params)

In [131]:
resp.raise_for_status()

In [132]:
income = resp.json()

In [133]:
income['income'][0].keys()

dict_keys(['costOfRevenue', 'currency', 'ebit', 'filingType', 'fiscalDate', 'fiscalQuarter', 'fiscalYear', 'grossProfit', 'incomeTax', 'interestIncome', 'minorityInterest', 'netIncome', 'netIncomeBasic', 'operatingExpense', 'operatingIncome', 'otherIncomeExpenseNet', 'pretaxIncome', 'reportDate', 'researchAndDevelopment', 'sellingGeneralAndAdmin', 'symbol', 'totalRevenue', 'id', 'key', 'subkey', 'updated'])

In [129]:
income

{'symbol': 'AAPL',
 'income': [{'costOfRevenue': 52868179751,
   'currency': 'USD',
   'ebit': 28650967267,
   'filingType': '10-Q',
   'fiscalDate': '2021-03-15',
   'fiscalQuarter': 2,
   'fiscalYear': 2037,
   'grossProfit': 39470002610,
   'incomeTax': 4420671152,
   'interestIncome': 673184408,
   'minorityInterest': 0,
   'netIncome': 24792002251,
   'netIncomeBasic': 24088867486,
   'operatingExpense': 64539222875,
   'operatingIncome': 28302697541,
   'otherIncomeExpenseNet': 0,
   'pretaxIncome': 28615249385,
   'reportDate': '2021-04-24',
   'researchAndDevelopment': 5406485745,
   'sellingGeneralAndAdmin': 5515487425,
   'symbol': 'AAPL',
   'totalRevenue': 91778501657,
   'id': 'CMNEIO',
   'key': 'ALAP',
   'subkey': 'quarterly',
   'updated': 1639636147569}]}

In [134]:
rev = []
rev_qrt = {}

for num in range(0,len(income['income'])):
    for key in ('fiscalDate','fiscalQuarter','totalRevenue'):
        rev_qrt[key] = income['income'][num].setdefault(key)
    rev.append(rev_qrt)
    rev_qrt = {}
        
rev_df = pd.DataFrame(rev)    
rev_df

In [16]:
# If you plan to use this same endpoint a few times, it makes sense to create a function to avoid repeating code. Here is an example:

In [ ]:
'/stock/aapl/income/10-Q?last=12'
resp = requests.get(sandbox_url+endpoint, params = sandbox_params)

In [138]:
def revenue_history(_symbol, _params, _last=None):
    endpoint = f'{sandbox_url}/stock/{_symbol}/income/10-Q'
    if _last:
        endpoint += f'/?last={_last}'
    
    resp = requests.get(endpoint, params=_params)
    resp.raise_for_status()
    resp = resp.json()
    
    rev = []
    rev_qrt = {}

    for num in range(0,len(resp['income'])):
        for key in ('fiscalDate','fiscalQuarter','totalRevenue'):
            rev_qrt[key] = resp['income'][num].setdefault(key)
        rev.append(rev_qrt)
        rev_qrt = {}
    
    return pd.DataFrame(rev)

In [140]:
appl_2qrt_df = revenue_history('AAPL', sandbox_params, 2)
appl_2qrt_df

,fiscalDate,fiscalQuarter,totalRevenue
0,2021-03-24,2,92332286186
1,2020-12-16,1,111978134657


In [17]:
# def historical_data(_symbol, _range=None, _date=None):
#     endpoint = f'{base_url}/stock/{_symbol}/chart'
#     if _range:
#         endpoint += f'/{_range}'
#     elif _date:
#         endpoint += f'/date/{_date}'
    
#     resp = requests.get(endpoint, params=params)
#     resp.raise_for_status()
#     return pd.DataFrame(resp.json())

In [26]:
#appl_3m_df = historical_data('AAPL', _range='1m')